INF8770 Technologies multimédias

Polytechnique Montréal

Importation des librairies

In [ ]:
from PIL import Image
import os
from skimage.feature.texture import local_binary_pattern
from skimage.feature import graycomatrix
import glob
import cv2
import numpy
import csv
import time

Importation des données

In [ ]:
query = glob.glob(os.path.join("./data", "*.jpg"))
database = glob.glob(os.path.join("./data/database", "*.jpg"))

query_cropped = glob.glob(os.path.join("./data/cropped", "*.jpg"))
database_cropped = glob.glob(os.path.join("./data/cropped/database", "*.jpg"))

Background Substraction

In [ ]:
def desc_cm_params(imagePath, distances = [1, 5, 10], angles = [0, numpy.pi / 2, numpy.pi, 3 * numpy.pi / 4]):
    image = cv2.imread(imagePath, cv2.IMREAD_COLOR)
    nb_channels = 3
    nb_rows = len(image)
    nb_columns = len(image[0])

    three_channel_one_dimension_desc = []
    
    for channel in range(nb_channels):
      one_channel_image = []
      for row in range(nb_rows):
        one_channel_image.append([])
        for column in range(nb_columns):
          one_channel_image[row].append(image[row][column][channel])
      cm = graycomatrix(one_channel_image, distances, angles, normed=True)
    
      one_dimension_descriptors = []
      for distance in range(len(cm[0][0])):
        one_dimension_descriptors.append([])
        for angle in range(len(cm[0][0][0])):
            one_dimension_descriptors[distance].append([])
            one_dim_desc = []
            for row in range(len(cm)):
                for col in range(len(cm[0])):
                    one_dim_desc.append(cm[row][col][distance][angle])
            one_dimension_descriptors[distance][angle].append(numpy.asarray(one_dim_desc, numpy.float32))
      three_channel_one_dimension_desc.append(one_dimension_descriptors)

    return three_channel_one_dimension_desc

Instance Segmentation

In [ ]:
def desc_LBP(path, longueurCodes, rayon):
  image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
  desc = local_binary_pattern(image, longueurCodes, rayon)
  return numpy.asarray(desc, numpy.float32)

Méthode de comparaison

In [ ]:
def intersection(vector_i, vector_j):
    return numpy.sum(numpy.minimum(vector_i, vector_j)) / numpy.sum(vector_j)

def norms1(vector_i, vector_j):
    return numpy.sum(numpy.abs(vector_i - vector_j))

def norms2(vector_i, vector_j):
    return numpy.sqrt(numpy.sum(numpy.power(vector_i - vector_j, 2)))

def bhattacharyya(vector_i, vector_j):
    return -numpy.log((numpy.sum(numpy.sqrt(numpy.multiply(vector_i, vector_j)))))

Classification par matrice de co-occurrence

In [ ]:
def classify_CM(query, database):
    cm_db_descriptors = []
    for path_db in database:
        cm_db_descriptors.append(desc_cm_params(path_db))
    
    for path_query in query:
        result = dict()

        cm_query_descriptors = desc_cm_params(path_query)
    
        methods = ["intersection", "norms1", "norms2", "bhattacharyya"]

        #path = path_query.split('\\')[1][:-4]
        # with open(f"./results/CM/{path}.csv", 'w+', newline='') as file:
        #     writer = csv.writer(file)
        #     field = [path] + [x.split('\\')[1][:-4] for x in database]
            
        for key in methods:
            result[key] = []

        index = 0
        for path_db in database:
            result_id = path_query.split('\\')[1][:-4] + ":" + path_db.split('\\')[1][:-4]

            intersectionResult = 0
            norms1Result = 0
            norms2Result = 0
            bhattacharyyaResult = 0
            for channel in range(len(cm_query_descriptors)): # 3 channels
                for distance in range(len(cm_query_descriptors[0])): # 3 distances
                    for angle in range(len(cm_query_descriptors[0][0])): # 4 angles
                        intersectionResult += intersection(numpy.asarray(cm_query_descriptors[channel][distance][angle]), numpy.asarray(cm_db_descriptors[index][channel][distance][angle]))
                        norms1Result += norms1(numpy.asarray(cm_query_descriptors[channel][distance][angle]), numpy.asarray(cm_db_descriptors[index][channel][distance][angle]))
                        norms2Result += norms2(numpy.asarray(cm_query_descriptors[channel][distance][angle]), numpy.asarray(cm_db_descriptors[index][channel][distance][angle]))
                        bhattacharyyaResult += bhattacharyya(numpy.asarray(cm_query_descriptors[channel][distance][angle]), numpy.asarray(cm_db_descriptors[index][channel][distance][angle]))
            result["intersection"].append((result_id, intersectionResult))
            result["norms1"].append((result_id, norms1Result))
            result["norms2"].append((result_id, norms2Result))
            result["bhattacharyya"].append((result_id, bhattacharyyaResult))

            index += 1
                
            # writer.writerow([path + f" ({timeList[0]})"] + methods + ["Time"])

            # for f in range(1, len(field)):
            #     writer.writerow([field[f]] + [result[key][f - 1][1] for key in result.keys()])

timestamp = time.time()
classify_CM(query, database)
print(f"Time Normal: {time.time() - timestamp}")

timestamp = time.time()
classify_CM(query_cropped, database_cropped)
print(f"Time Cropped: {time.time() - timestamp}")

Classification par modèle binaire locaux

In [ ]:
def classify_LBP(query, database, longueurCodes, rayon, all):
    lbp_db_descriptors = []
    for path_db in database:
        row_lbp, col_lbp = cv2.imread(path_db, cv2.IMREAD_GRAYSCALE).shape
        lbp_hist_db, _ = numpy.histogram(desc_LBP(path_db, longueurCodes, rayon), bins = range(256))
        lbp_hist_db = lbp_hist_db / (row_lbp * col_lbp)
        lbp_db_descriptors.append(lbp_hist_db)

    for path_query in query:
        result = dict()

        lbp_query = desc_LBP(path_query, longueurCodes, rayon)

        row_lbp, col_lbp = cv2.imread(path_query, cv2.IMREAD_GRAYSCALE).shape

        lbp_hist_query, _ = numpy.histogram(lbp_query, bins = range(256))
        lbp_hist_query = lbp_hist_query / (row_lbp * col_lbp)

        methods = ["intersection", "norms1", "norms2", "bhattacharyya"]

        # path = path_query.split('\\')[1][:-4]
        # result_path = f"./results/LBP/R{rayon}-P{longueurCodes}"

        # os.makedirs(result_path, exist_ok=True)

        # with open(f"{result_path}/{path}.csv", 'w+', newline='') as file:
            # writer = csv.writer(file)
            # field = [path] + [x.split('\\')[1][:-4] for x in database]
            
        for key in methods:
            result[key] = []
        
        index = 0
        for path_db in database:
            lbp_hist_db = lbp_db_descriptors[index]

            result_id = path_query.split('\\')[1][:-4] + ":" + path_db.split('\\')[1][:-4]

            result["intersection"].append((result_id, intersection(lbp_hist_query, lbp_hist_db)))

            result["norms1"].append((result_id, norms1(lbp_hist_query, lbp_hist_db)))

            result["norms2"].append((result_id, norms2(lbp_hist_query, lbp_hist_db)))

            result["bhattacharyya"].append((result_id, bhattacharyya(lbp_hist_query, lbp_hist_db)))

            index += 1

            # writer.writerow([path + f" ({timeList[0]})"] + methods)

            # for f in range(1, len(field)):
            #     writer.writerow([field[f]] + [result[key][f - 1][1] for key in result.keys()])
        
        # for key in result.keys():
        #     if key not in all:
        #         all[key] = dict()
        #     reverse = True if (key == "intersection") else False
        #     data = [x[0] for x in sorted(result[key], key=lambda tp: tp[1], reverse=reverse)[:3]]
        #     for d in data:
        #         img_name = d.split(':')[0][:-6]
        #         db_name = d.split(':')[1][:-2]
        #         if "strawberry" in img_name:
        #             if img_name not in all[key]:
        #                 all[key][img_name] = dict()
        #             if db_name not in all[key][img_name]:
        #                 all[key][img_name][db_name] = 0
        #             all[key][img_name][db_name] += 1
        #         else:
        #             if img_name not in all[key]:
        #                 all[key][img_name] = (0, 0)
        #             if img_name == db_name:
        #                 all[key][img_name] = (all[key][img_name][0] + 1, all[key][img_name][1])
        #             all[key][img_name] = (all[key][img_name][0], all[key][img_name][1] + 1)

all = dict()

timestamp = time.time()
for rayon in range(1, 6, 2):
    for longueurCodes in range(4, 25, 4):
        classify_LBP(query, database, longueurCodes, rayon, all)

print(f"Time Normal: {time.time() - timestamp}")

timestamp = time.time()
for rayon in range(1, 6, 2):
    for longueurCodes in range(4, 25, 4):
        classify_LBP(query_cropped, database_cropped, longueurCodes, rayon, all)
print(f"Time Cropped: {time.time() - timestamp}")

# print(all)